<a href="https://colab.research.google.com/github/Andre-Williams22/Reinforcement-Learning-Agent/blob/volatile-stock/most_volatile_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install alpaca_trade_api

In [ ]:
!pip3 install yfinance

In [31]:
import requests
import json as json
from datetime import datetime
import alpaca_trade_api as t
import logging
import pytz
import time
import csv
# input : year(date), num of stocks
# output: list of volatile stocks

import yfinance as yf
# from pandas_datareader import data as pdr
import pandas as pd

In [21]:

def get_stock_symbols():
  sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
  df = sp500[0]
  symbols = df['Symbol'].tolist()
  return symbols

In [22]:
print(get_stock_symbols())

['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BK', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'BIIB', 'BLK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF.B', 'CHRW', 'COG', 'CDNS', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CERN', 'CF', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'CCI', 'CSX', 'CMI', 'CVS', 'DHI', 'DHR', 'DRI', 'DVA', 'DE', 'DAL', 'XRAY

In [32]:
def df_lookup(df, key_row, key_col):
  try:
    return df.iloc[key_row][key_col]
  except IndexError:
    return 0

In [36]:
def get_movement_list(stocks, period):
  movement_list = []
  f = open("stock_changes.csv", "w")
  stock_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
  stock_writer.writerow(["stock", "delta_percent", "delta_price"]) # add header to csv
  for stock in stocks:
    # get history
    curr_stock = yf.Ticker(stock)
    hist = curr_stock.history(period = period) #lookback 1 day

    low = float(10000)
    high = float(0)
    # print(curr_stock.info)
    for day in hist.itertuples(index=True, name='Pandas'):
      if day.Low < low:
        low = day.Low
      if high < day.High:
        high = day.High
    #for zero division error handling
    # if low == 0:
    #   delta_percent = 0
    # else:
    delta_percent = 100 * (high - low) / low #check for division by 0
    Open = df_lookup(hist, 0, "Open")

    # some error handling: 
    if len(hist >= 5):
      Close = df_lookup(hist, 4, "Close")
    else :
      Close = Open

    if (Open == 0):
      delta_price = 0
    else:
      delta_price = 100 * (Close - Open) / Open 
 
    print(stock+" "+str(delta_percent)+ " "+ str(delta_price))
    pair = [stock, delta_percent, delta_price]
    movement_list.append(pair)
    stock_writer.writerow(pair)
  #close the txt file
  f.close()
  return movement_list   

In [37]:
# time get movement list function #started 6:36, end 6:40
stocks = get_stock_symbols()
start = time.perf_counter()
get_movement_list(stocks, "1d")
end = time.perf_counter()
print(end - start)

MMM 1.2242289091660123 -100.0
ABT 2.908903783127143 -100.0
ABBV 1.4671610601389333 -100.0
ABMD 2.5533385901147017 -100.0
ACN 3.042606921673049 -100.0
ATVI 1.767653164235297 -100.0
ADBE 1.4504451217150802 -100.0
AMD 2.255445395794403 -100.0
AAP 3.149712644161471 -100.0
AES 2.49382305895021 -100.0
AFL 1.4398412845191537 -100.0
A 1.6377235032375794 -100.0
APD 3.713541245397744 -100.0
AKAM 2.0261929009468505 -100.0
ALK 5.027851427441574 -100.0
ALB 3.210043124762672 -100.0
ARE 2.0707194328184153 -100.0
ALXN 0.9376895044855991 -100.0
ALGN 4.052469150490404 -100.0
ALLE 1.6872674761692945 -100.0
LNT 2.2887363225526594 -100.0
ALL 2.5039832906669117 -100.0
GOOGL 1.7010052960131712 -100.0
GOOG 1.7505226472404134 -100.0
MO 2.0442790952457033 -100.0
AMZN 2.00967300810546 -100.0
AMCR 11.374638910097039 -100.0
AEE 1.5384576259515224 -100.0
AAL 5.150760020150164 -100.0
AEP 2.597883813037304 -100.0
AXP 1.577411420412821 -100.0
AIG 2.2723338187732094 -100.0
AMT 1.6579319474277643 -100.0
AWK 1.8911985364

In [52]:
def get_highest_movers():
  #read the stock_changes csv file
  stocks = pd.read_csv('stock_changes.csv')
  #sort by delta percent
  sorted_stocks = stocks.sort_values('delta_percent', ascending=False)
  # #take the top 20 values
  most_volatile_stocks = sorted_stocks.head(20)
  return most_volatile_stocks['stock'].tolist()

In [53]:
get_highest_movers()

['AMCR',
 'CCL',
 'ETSY',
 'OXY',
 'NCLH',
 'FLS',
 'SIVB',
 'V',
 'DG',
 'MCHP',
 'FANG',
 'ENPH',
 'MRO',
 'BBY',
 'CB',
 'APA',
 'XRX',
 'NKE',
 'DISCK',
 'DISCA']